With column transformer we can use multiple transformations in one line of code

In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [6]:
X_train

,age,gender,fever,cough,city
75,5,Male,102.0,Mild,Kolkata
74,34,Female,104.0,Strong,Delhi
53,83,Male,98.0,Mild,Delhi
60,24,Female,102.0,Strong,Bangalore
92,82,Female,102.0,Strong,Kolkata
...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore
63,10,Male,100.0,Mild,Bangalore
88,5,Female,100.0,Mild,Kolkata
37,55,Male,100.0,Mild,Kolkata


## 1. Not using column transformer

In [83]:
# adding simple imputer to fever col
si = SimpleImputer()   ##simple imputer() is to replace the missing values with mean
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [10]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [87]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [89]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [92]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## 2. Using column transformer

In [1]:
from sklearn.compose import ColumnTransformer

- column transformer takes list of transformers and remainder as parameters
- we applied 3 transformers: SimpleImputer, oe(oridinal encoding), ohe
- tnfs are the names that we gave to transformers
- what we want to do to the columns where we don't wanna use transformers can be specified using remainder.
- It can be either drop(drops the remaining columns) or passthrough(leaves the remaining columns as it is)

In [9]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')
# sparse matrix or sparse array is a matrix in which most of the
#  elements are zeros

In [13]:
transformer.fit_transform(X_train)

array([[102.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  34.        ],
       [ 98.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  83.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  24.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  69.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  47.        ],
       [100.97183099,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.    

In [14]:
transformer.transform(X_test).shape

(20, 7)